<a href="https://colab.research.google.com/github/wenqiglantz/hands-on-llamaindex/blob/main/02_agents_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAIAgent

`OpenAIAgent` is an OpenAI (function calling) Agent. It uses the OpenAI function API to reason about whether to use a tool, and returning the response to the user. It supports both a flat list of tools as well as retrieval over the tools.

LlamaIndex notebook: https://docs.llamaindex.ai/en/stable/examples/agent/openai_agent_with_query_engine.html.

## Step 1: Install and Setup

In [1]:
!pip install -q llama_index pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.0/631.0 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

In [2]:
import logging, sys, os
import nest_asyncio
from google.colab import userdata

# set OpenAI API key in environment variable
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# serves to enable nested asynchronous event loops, recommended for colab notebook
nest_asyncio.apply()

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
!mkdir reports
!wget https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2020/executive-summary-2020.pdf -O ./reports/2020-executive-summary.pdf
!wget https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2021/executive-summary-2021.pdf -O ./reports/2021-executive-summary.pdf
!wget https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2022/executive-summary-2022.pdf -O ./reports/2022-executive-summary.pdf

--2024-02-16 17:07:00--  https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2020/executive-summary-2020.pdf
Resolving www.fiscal.treasury.gov (www.fiscal.treasury.gov)... 166.123.218.167, 2610:108:4100:100c::8:118
Connecting to www.fiscal.treasury.gov (www.fiscal.treasury.gov)|166.123.218.167|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2323072 (2.2M) [application/pdf]
Saving to: ‘./reports/2020-executive-summary.pdf’

./reports/2020-exec 100%[===================>]   2.21M  1.13MB/s    in 2.0s    

2024-02-16 17:07:02 (1.13 MB/s) - ‘./reports/2020-executive-summary.pdf’ saved [2323072/2323072]

--2024-02-16 17:07:02--  https://www.fiscal.treasury.gov/files/reports-statements/financial-report/2021/executive-summary-2021.pdf
Resolving www.fiscal.treasury.gov (www.fiscal.treasury.gov)... 166.123.218.167, 2610:108:4100:100c::8:118
Connecting to www.fiscal.treasury.gov (www.fiscal.treasury.gov)|166.123.218.167|:443... connected.
HTTP reque

## Step 2: Load data, build indices, define OpenAIAgent

In [5]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.agent.openai import OpenAIAgent
import os

query_engine_tools = []

for filename in os.listdir("reports"):
    if filename.endswith(".pdf"):
        file_path = os.path.join("reports", filename)

        with open(file_path, "r") as file:
            documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
            print(f"Loaded {len(documents)} documents from {filename}")
            print(filename[:-4])

            index = VectorStoreIndex.from_documents(documents)
            query_engine = index.as_query_engine(similarity_top_k=5)
            query_engine_tool = QueryEngineTool.from_defaults(
                query_engine=query_engine,
                name=f"{filename[:-4]}",  # Construct name without extension
                description=f"Provides information about the U.S. government financial report {filename[:-4]}",
            )
            query_engine_tools.append(query_engine_tool)

agent = OpenAIAgent.from_tools(query_engine_tools, verbose=True)

Loaded 11 documents from 2021-executive-summary.pdf
2021-executive-summary
Loaded 11 documents from 2020-executive-summary.pdf
2020-executive-summary
Loaded 10 documents from 2022-executive-summary.pdf
2022-executive-summary


## Step 3: Execute Queries

In [6]:
from IPython.display import Markdown

response = agent.chat("Can you compare and contrast the government's bottom line net operating cost amount for all three years and tell me which year has the highest cost?")
display(Markdown(f"<b>{response}</b>"))

Added user message to memory: Can you compare and contrast the government's bottom line net operating cost amount for all three years and tell me which year has the highest cost?
=== Calling Function ===
Calling function: 2021-executive-summary with args: {
  "input": "bottom line net operating cost"
}
Got output: The bottom line net operating cost refers to the government's total net cost after subtracting earned program revenues and adjusting for gains or losses from changes in actuarial assumptions. It represents the government's overall financial position and is calculated by subtracting tax and other revenues from the net cost.

=== Calling Function ===
Calling function: 2020-executive-summary with args: {
  "input": "bottom line net operating cost"
}
Got output: The bottom line net operating cost refers to the total net cost incurred by the government after subtracting earned program revenues and making adjustments for gains or losses from changes in actuarial assumptions. It rep

<b>To compare and contrast the government's bottom line net operating cost for all three years, we need to analyze the executive summaries for each year. Unfortunately, I don't have access to the specific information from the executive summaries at the moment. However, I can provide you with a general understanding of the concept.

The bottom line net operating cost represents the total net cost incurred by the government after considering earned program revenues and adjustments for gains or losses resulting from changes in actuarial assumptions. It reflects the overall financial position of the government's operations and programs.

To determine which year has the highest cost, we would need to compare the specific values from each year's executive summary. Without that information, it is not possible to determine the year with the highest cost.

If you have access to the executive summaries for each year, please provide the specific values, and I will be able to assist you further in comparing and identifying the year with the highest cost.</b>